# **Homework 2-1 Phoneme Classification**

## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**
**https://drive.google.com/file/d/1P9RfXu19Equ0VdCs803XGGr8rIIT4XqH/view?usp=sharing**


**https://hackmd.io/@lido2370/S1aX6e1nN?type=view**



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#/content/drive/MyDrive/windows備份/C_Users_Alvin_Documents/~碩班/Course/109-2/機器學習_李宏毅/Week 3/HW2

Mounted at /content/drive


In [ ]:
#!gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
!cp '/content/drive/MyDrive/windows備份/C_Users_Alvin_Documents/~碩班/Course/109-2/機器學習_李宏毅/Week 3/HW2/timit_11_v2.zip' './data.zip'
!unzip data.zip
!ls 

Archive:  data.zip
   creating: timit_11/
  inflating: timit_11/train_11.npy   
  inflating: timit_11/test_11.npy    
  inflating: timit_11/train_label_11.npy  
data.zip  drive  sample_data  timit_11


## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [ ]:
import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')
print('Loading done!')

Loading data ...
Loading done!


In [ ]:
print('Size of training data: {}'.format(train.shape))
print('Size of training data: {}'.format(train_label.shape))
print('Size of testing data: {}'.format(test.shape))

Size of training data: (1229932, 429)
Size of training data: (1229932,)
Size of testing data: (451552, 429)


## Create Dataset

In [ ]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int) # 數據轉換成int
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [ ]:
VAL_RATIO = 0.1 # 0表示全部資料都使用

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))


Size of training set: (1106938, 429)
Size of validation set: (122994, 429)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [ ]:
BATCH_SIZE = 128

from torch.utils.data import DataLoader

train_set = TIMITDataset(train, train_label)
val_set = TIMITDataset(val_x, val_y)
test_set = TIMITDataset(test, None)

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [ ]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

52

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.
* https://ensemble-pytorch.readthedocs.io/en/latest/

In [ ]:
import torch
import torch.nn as nn
class LSTMModel(nn.Module):
    def __init__(self):
        super(LSTMModel, self).__init__()
        self.hl = 512
        self.lstm = nn.LSTM(39, self.hl, 2, bidirectional=True)
        self.fc = nn.Linear(self.hl*11*2, 39)
    
    def forward(self, x):

        out = x.view(x.size(0), 11, 39) #128*11*39
        out = out.permute(1, 0, 2) #11*128*39

        h0 = Variable(torch.zeros(2*2, x.size(0), self.hl).cuda())
        c0 = Variable(torch.zeros(2*2, x.size(0), self.hl).cuda())

        out, (hn, cn) = self.lstm(out, (h0, c0)) # 11*128*512
        out = out.permute(1, 0, 2) 
        out = out.reshape(x.size(0), self.hl*11*2) #128*(5632*2)
        out = self.fc(out) 

        return out

## Training

In [ ]:
#check device
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Fix random seeds for reproducibility.

Feel free to change the training parameters here.

In [ ]:
!pip install torchensemble
from torchensemble import VotingClassifier
from torch.autograd import Variable
from torchensemble.utils.logging import set_logger

     |████████████████████████████████| 22.3MB 1.3MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
same_seeds(0)

# get device 
device = get_device()
print(f'DEVICE: {device}')
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
logger = set_logger("classification_mnist_mlp")
model = VotingClassifier(estimator=LSTMModel ,n_estimators=10, cuda=True).to(device)
optimizer = model.set_optimizer("Adam", lr=0.001, weight_decay=5e-6)
model.fit(train_loader, epochs=2, test_loader=val_loader)

DEVICE: cuda
Log will be saved in '/content/logs'.
Create folder 'logs/'
Start logging into file /content/logs/classification_mnist_mlp-2021_04_01_13_30.log...
Estimator: 000 | Epoch: 000 | Batch: 000 | Loss: 3.66240 | Correct: 1/128
Estimator: 000 | Epoch: 000 | Batch: 100 | Loss: 1.55847 | Correct: 71/128
Estimator: 000 | Epoch: 000 | Batch: 200 | Loss: 1.27966 | Correct: 74/128
Estimator: 000 | Epoch: 000 | Batch: 300 | Loss: 1.32083 | Correct: 73/128
Estimator: 000 | Epoch: 000 | Batch: 400 | Loss: 1.24615 | Correct: 74/128
Estimator: 000 | Epoch: 000 | Batch: 500 | Loss: 1.27234 | Correct: 78/128
Estimator: 000 | Epoch: 000 | Batch: 600 | Loss: 1.06892 | Correct: 85/128
Estimator: 000 | Epoch: 000 | Batch: 700 | Loss: 0.95281 | Correct: 89/128
Estimator: 000 | Epoch: 000 | Batch: 800 | Loss: 0.89643 | Correct: 91/128
Estimator: 000 | Epoch: 000 | Batch: 900 | Loss: 0.92905 | Correct: 87/128
Estimator: 000 | Epoch: 000 | Batch: 1000 | Loss: 0.94817 | Correct: 95/128
Estimator: 000 

2021-04-01 13:55:48,291 - INFO: Saving the model to `./VotingClassifier_LSTMModel_10_ckpt.pth`
2021-04-01 13:55:49,336 - INFO: Epoch: 000 | Validation Acc: 88.129 % | Historical Best: 88.129 %


Estimator: 000 | Epoch: 001 | Batch: 000 | Loss: 0.69060 | Correct: 98/128
Estimator: 000 | Epoch: 001 | Batch: 100 | Loss: 0.51055 | Correct: 104/128
Estimator: 000 | Epoch: 001 | Batch: 200 | Loss: 0.50971 | Correct: 97/128
Estimator: 000 | Epoch: 001 | Batch: 300 | Loss: 0.55351 | Correct: 103/128
Estimator: 000 | Epoch: 001 | Batch: 400 | Loss: 0.50944 | Correct: 105/128
Estimator: 000 | Epoch: 001 | Batch: 500 | Loss: 0.44622 | Correct: 108/128
Estimator: 000 | Epoch: 001 | Batch: 600 | Loss: 0.37885 | Correct: 110/128
Estimator: 000 | Epoch: 001 | Batch: 700 | Loss: 0.40231 | Correct: 111/128
Estimator: 000 | Epoch: 001 | Batch: 800 | Loss: 0.50619 | Correct: 105/128
Estimator: 000 | Epoch: 001 | Batch: 900 | Loss: 0.52658 | Correct: 102/128
Estimator: 000 | Epoch: 001 | Batch: 1000 | Loss: 0.56318 | Correct: 103/128
Estimator: 000 | Epoch: 001 | Batch: 1100 | Loss: 0.56759 | Correct: 98/128
Estimator: 000 | Epoch: 001 | Batch: 1200 | Loss: 0.45662 | Correct: 111/128
Estimator: 0

2021-04-01 14:21:06,542 - INFO: Saving the model to `./VotingClassifier_LSTMModel_10_ckpt.pth`
2021-04-01 14:21:07,623 - INFO: Epoch: 001 | Validation Acc: 94.648 % | Historical Best: 94.648 %


## Testing

Create a testing dataset, and load model from the saved checkpoint.

Make prediction.

In [ ]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [ ]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        if i < len(predict) - 1 and i > 0: # post processing
            if (predict[i-1] == predict[i+1] and y != predict[i-1]):
                y = predict[i-1]
            if (predict[i-1] != predict[i+1] and y != predict[i-1] and y != predict[i+1]):
                y = predict[i-1]
        f.write('{},{}\n'.format(i, y))